In [847]:
# Dependencies and Setup
from pathlib import Path
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")

In [848]:
school_data_complete

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12,Thomas High School,51,48,14,Independent,1635,1043130
39166,39166,Dawn Bell,F,10,Thomas High School,81,89,14,Independent,1635,1043130
39167,39167,Rebecca Tanner,F,9,Thomas High School,99,99,14,Independent,1635,1043130
39168,39168,Desiree Kidd,F,10,Thomas High School,72,77,14,Independent,1635,1043130


In [849]:
# Local Government Area (LGA) Summary
# unique schools count
school_count = school_data_complete['school_name'].nunique()

In [850]:
# students count
student_count = school_data_complete['student_name'].count()

In [851]:
# calc total budget
schools_budget = school_data_complete.drop_duplicates(subset='school_name').reset_index(drop=True)['budget'].sum()

In [852]:
average_maths_score = school_data_complete['maths_score'].mean()

In [853]:
average_reading_score = school_data_complete['reading_score'].mean()

In [854]:
passing_maths_count = school_data_complete[(school_data_complete["maths_score"] >= 50)].count()["student_name"]
passing_maths_percentage = passing_maths_count / float(student_count) * 100

In [855]:
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 50)].count()["student_name"]
passing_reading_percentage = passing_reading_count/ float(student_count) * 100

In [856]:
students_passing_math = school_data_complete[(school_data_complete["maths_score"] >= 50)]
students_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 50)]
intersection_count = pd.merge(students_passing_math, students_passing_reading, how='inner').count()["student_name"]

In [857]:
# percentage pf students passing both reading & math
overall_passing_percentage = intersection_count/ float(student_count) * 100

In [858]:
# LGA Summary
LGA_summary_map= {
    'Total Schools': [school_count],
    'Total Students': [student_count],
    'Total Budget': [schools_budget],
    'Average Maths Score': [average_maths_score],
    'Average Reading Score': [average_reading_score],
    '% Passing Maths': [passing_maths_percentage],
    '% Passing Reading': [passing_reading_percentage],
    '% Overall Passing': [overall_passing_percentage]
}

LGA_summary_data = pd.DataFrame(LGA_summary_map)

In [859]:
# Local Government Area (LGA) Summary
LGA_summary_data

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,24649428,70.338192,69.980138,86.078632,84.426857,72.808272


In [860]:
# School Summary work below

In [861]:
# Use the code provided to select the type per school from school_data
school_types = school_data.set_index(["school_name"])["type"].reset_index(name='type')

# Calculate the total student count per school
per_school_counts = student_data.groupby(["school_name"]).size().reset_index(name='student_count')

# Calculate the total school budget and per capita spending per school from school_data
per_school_budget = school_data.set_index(["school_name"])["budget"].reset_index(name='school_budget')

df_merged = pd.merge(per_school_counts, per_school_budget, on='school_name')
df_merged['per_capita_budget'] = df_merged['school_budget'] / df_merged['student_count']
per_capita_budget = df_merged[['school_name', 'per_capita_budget']]

# Calculate the average test scores per school from school_data_complete
per_school_maths = school_data_complete.groupby('school_name')['maths_score'].mean().reset_index()
per_school_maths.columns = ['school_name', 'average_maths_score']

per_school_reading = school_data_complete.groupby('school_name')['reading_score'].mean().reset_index()
per_school_reading.columns = ['school_name', 'average_reading_score']

In [862]:
# Get the students who passed maths and passed reading by creating separate filtered DataFrames from school_data_complete.
total_students_per_school = school_data_complete.groupby('school_name').size()
students_passing_math = school_data_complete[school_data_complete['maths_score'] >= 50].groupby('school_name').size()
school_passing_maths = (students_passing_math / total_students_per_school * 100).reset_index(name='math_passing_percentage')

students_passing_reading = school_data_complete[school_data_complete['reading_score'] >= 50].groupby('school_name').size()
school_passing_reading = (students_passing_reading / total_students_per_school * 100).reset_index(name='reading_passing_percentage')

# Get the students who passed both reading and maths in a separate DataFrame from school_data_complete.
students_passing_maths_and_reading = school_data_complete[(school_data_complete['reading_score'] >= 50) & (school_data_complete['maths_score'] >= 50)]
school_overall_passing_rate = (students_passing_maths_and_reading.groupby('school_name').size() / total_students_per_school * 100).reset_index(name='overall_passing_rate')

In [863]:
# School Summary
school_summary_data = pd.merge(school_types, per_school_counts, on='school_name')
school_summary_data = pd.merge(school_summary_data, per_school_budget, on='school_name')
school_summary_data = pd.merge(school_summary_data, per_capita_budget, on='school_name')
school_summary_data = pd.merge(school_summary_data, per_school_maths, on='school_name')
school_summary_data = pd.merge(school_summary_data, per_school_reading, on='school_name')
school_summary_data = pd.merge(school_summary_data, school_passing_reading, on='school_name')
school_summary_data = pd.merge(school_summary_data, school_passing_maths, on='school_name')
school_summary_data = pd.merge(school_summary_data, school_overall_passing_rate, on='school_name')
school_summary_data['school_budget'] = school_summary_data['school_budget'].apply(lambda x: '${:,.2f}'.format(x))
school_summary_data['per_capita_budget'] = school_summary_data['per_capita_budget'].apply(lambda x: '${:,.2f}'.format(x))


In [864]:
school_summary_data

,school_name,type,student_count,school_budget,per_capita_budget,average_maths_score,average_reading_score,reading_passing_percentage,math_passing_percentage,overall_passing_rate
0,Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.453548,81.693521,66.712376
1,Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,82.807731,81.654798,67.650051
2,Shelton High School,Independent,1761,"$1,056,600.00",$600.00,72.034072,70.257808,86.712095,91.538898,78.875639
3,Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,81.877023,80.949299,66.364617
4,Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,88.487738,91.212534,81.335150
5,Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,81.296540,82.785808,67.455103
6,Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,89.074273,90.850377,80.785791
7,Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,87.379421,91.639871,80.084405
8,Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,88.524590,89.929742,78.922717
9,Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,86.590437,91.683992,79.209979


In [865]:
# Highest-Performing Schools (by % Overall Passing)
highest_performing_schools_by_overall_passing =  school_summary_data.sort_values(by = 'overall_passing_rate', ascending = False)
highest_performing_schools_by_overall_passing.head(5)

,school_name,type,student_count,school_budget,per_capita_budget,average_maths_score,average_reading_score,reading_passing_percentage,math_passing_percentage,overall_passing_rate
4,Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,88.487738,91.212534,81.335150
6,Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,89.074273,90.850377,80.785791
7,Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,87.379421,91.639871,80.084405
10,Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,86.666667,91.777778,79.722222
11,Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,87.396849,90.797699,79.419855


In [866]:
lowest_performing_schools_by_overall_passing =  school_summary_data.sort_values(by = 'overall_passing_rate', ascending = True)
lowest_performing_schools_by_overall_passing.head(5)

,school_name,type,student_count,school_budget,per_capita_budget,average_maths_score,average_reading_score,reading_passing_percentage,math_passing_percentage,overall_passing_rate
3,Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,81.877023,80.949299,66.364617
0,Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.453548,81.693521,66.712376
12,Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,81.978576,82.062592,67.191766
5,Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,81.296540,82.785808,67.455103
13,Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.219788,82.438846,67.469880


In [867]:
# Maths Scores by Year
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

average_maths_scores_9 = year_nine.groupby('school_name', as_index=False)['maths_score'].mean()
average_maths_scores_9.rename(columns={'maths_score': 'year 9'}, inplace=True)
average_maths_scores_10 = year_ten.groupby('school_name', as_index=False)['maths_score'].mean()
average_maths_scores_10.rename(columns={'maths_score': 'year 10'}, inplace=True)
average_maths_scores_11 = year_eleven.groupby('school_name', as_index=False)['maths_score'].mean()
average_maths_scores_11.rename(columns={'maths_score': 'year 11'}, inplace=True)
average_maths_scores_12 = year_twelve.groupby('school_name', as_index=False)['maths_score'].mean()
average_maths_scores_12.rename(columns={'maths_score': 'year 12'}, inplace=True)

In [868]:
combined_average_maths = pd.merge(average_maths_scores_9, average_maths_scores_10, on='school_name')
combined_average_maths = pd.merge(combined_average_maths, average_maths_scores_11, on='school_name')
combined_average_maths = pd.merge(combined_average_maths, average_maths_scores_12, on='school_name')
combined_average_maths

,school_name,year 9,year 10,year 11,year 12
0,Bailey High School,72.493827,71.897498,72.374900,72.675097
1,Cabrera High School,72.321970,72.437768,71.008299,70.604712
2,Figueroa High School,68.477804,68.331586,68.811001,69.325282
3,Ford High School,69.021609,69.387006,69.248862,68.617811
4,Griffin High School,72.789731,71.093596,71.692521,71.469178
5,Hernandez High School,68.586831,68.867156,69.154412,68.985075
6,Holden High School,70.543307,75.105263,71.640777,73.409639
7,Huang High School,69.081754,68.533246,69.431345,68.639316
8,Johnson High School,69.469286,67.990220,68.637730,69.287393
9,Pena High School,71.996364,72.396000,72.523438,71.187845


In [869]:
# Reading Scores by Year
average_reading_scores_9 = year_nine.groupby('school_name', as_index=False)['reading_score'].mean()
average_reading_scores_9.rename(columns={'reading_score': 'year 9'}, inplace=True)
average_reading_scores_10 = year_ten.groupby('school_name', as_index=False)['reading_score'].mean()
average_reading_scores_10.rename(columns={'reading_score': 'year 10'}, inplace=True)
average_reading_scores_11 = year_eleven.groupby('school_name', as_index=False)['reading_score'].mean()
average_reading_scores_11.rename(columns={'reading_score': 'year 11'}, inplace=True)
average_reading_scores_12 = year_twelve.groupby('school_name', as_index=False)['reading_score'].mean()
average_reading_scores_12.rename(columns={'reading_score': 'year 12'}, inplace=True)

combined_average_reading = pd.merge(average_reading_scores_9, average_reading_scores_10, on='school_name')
combined_average_reading = pd.merge(combined_average_reading, average_reading_scores_11, on='school_name')
combined_average_reading = pd.merge(combined_average_reading, average_reading_scores_12, on='school_name')
combined_average_reading

,school_name,year 9,year 10,year 11,year 12
0,Bailey High School,70.901920,70.848265,70.317346,72.195525
1,Cabrera High School,71.172348,71.328326,71.201245,71.856021
2,Figueroa High School,70.261682,67.677588,69.152327,69.082126
3,Ford High School,69.615846,68.988701,70.735964,68.849722
4,Griffin High School,72.026895,70.746305,72.385042,69.434932
5,Hernandez High School,68.477569,70.621842,68.418199,69.244136
6,Holden High School,71.598425,71.096491,73.310680,70.481928
7,Huang High School,68.670616,69.516297,68.740638,68.671795
8,Johnson High School,68.719286,69.295029,69.969115,67.992521
9,Pena High School,70.949091,72.324000,71.703125,71.513812
